# LDED Audiovisual Fusion 

Author: Chen Lequn.
Created on 31 Oct 2023.


### Notebook 4b: ML modelling - ablation study (feature-based audio visual fusion model)
- Full Dataset: All features (from 4a)
- Ablated Dataset 1: Features after removing those with Pearson correlation > 0.9 (within each modality).
- Ablated Dataset 2: Features from Ablated Dataset 1 with MI > 0.05.
- Ablated Dataset 3: Features from Ablated Dataset 1 with MI > 0.1.
- Ablated Dataset 4: Audio Features from All features.
- Ablated Dataset 5: Vision Features from All features.
- Ablated Dataset 6: Audio Features from Ablated Dataset 1.
- Ablated Dataset 7: Vision Features from Ablated Dataset 1.
- Ablated Dataset 8: Audio Features from Ablated Dataset 2.
- Ablated Dataset 9: Audio Features from Ablated Dataset 3.

In [1]:
# from sklearnex import patch_sklearn, config_context
# import dpctl
# patch_sklearn()

In [2]:
import wandb

In [3]:
import os
import glob
import sys
import math
import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy as sp
import itertools
from collections import defaultdict
from pprint import pprint
import pickle
from joblib import dump, load

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
from pylab import *
import seaborn as sns
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
%matplotlib inline

# Scikit-learn general
from sklearn import preprocessing, metrics, svm, datasets, tree, neighbors
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_validate, KFold, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, scale
from sklearn.ensemble import HistGradientBoostingClassifier

# Scikit-learn models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR
from sklearn.multiclass import OneVsRestClassifier

# Imbalanced learn
from imblearn.ensemble import BalancedRandomForestClassifier, RUSBoostClassifier

# XGBoost and LightGBM
import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

# Utilities
from scipy.stats import uniform, randint
from scipy.interpolate import griddata
from sklearn.utils import shuffle, resample, class_weight
from utils import *


In [4]:
PROJECT_ROOT_DIR = "../"
IMAGE_PATH = os.path.join(PROJECT_ROOT_DIR, "result_images", 'defect classification', 'class_name_v3', 'ML-baseline', 'ablation study')
model_path = os.path.join(PROJECT_ROOT_DIR, "trained_models", 'defect classification', 'class_name_v3', 'ML-baseline', 'ablation study')
scaler_path = os.path.join(PROJECT_ROOT_DIR, "saved_scalers", 'defect classification', 'class_name_v3', 'ML-baseline', 'ablation study')
os.makedirs(IMAGE_PATH, exist_ok=True)
os.makedirs(model_path, exist_ok=True)
os.makedirs(scaler_path, exist_ok=True)

Multimodal_dataset_PATH = "/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset"
Dataset_path = os.path.join(Multimodal_dataset_PATH, f'25Hz')
                            

## function for automatically save the diagram/graph into the folder 
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 2.50

In [5]:
# Initialize wandb
wandb.init(project="AudioVisual_ML_ablation_study_v2")

wandb: Currently logged in as: chen1189 (lequn). Use `wandb login --relogin` to force relogin


## Step 1: Dataset preparation

In [6]:
# Read the data back from the HDF5 file into a new DataFrame
df_multimodal = pd.read_hdf(os.path.join(Dataset_path, 'spatiotemporal_fused_multimodal.h5'), key='df')
df_multimodal = df_multimodal.dropna(subset=['class_name_v3'])
df_multimodal

,sample index,Time_Stamps,audio_file_name,image_file_name,class_name,class_name_v2,Layer number,Sample number,class_name_v3,rms_energy,...,center_y,contour_area,contour_length,X,Y,Z,Vx,Vy,Vz,Speed
0,1,0.00,sample_21_1.wav,sample_21_1.jpg,Laser-off,Laser-off,1.0,21,Laser-off,0.009018,...,0.000000,0.0,0.000000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000
1,2,0.04,sample_21_2.wav,sample_21_2.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.019286,...,239.000000,305442.0,2234.000000,0.000,-0.000,0.001,-0.074,-0.074,0.000,0.105
2,3,0.08,sample_21_3.wav,sample_21_3.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.019593,...,239.000000,305442.0,2234.000000,-0.010,0.015,-0.007,-0.051,0.026,-0.051,0.077
3,4,0.12,sample_21_4.wav,sample_21_4.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.030937,...,230.264496,291865.5,2279.781744,-0.007,0.016,0.010,0.173,-0.222,2.520,2.536
4,5,0.16,sample_21_5.wav,sample_21_5.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.038329,...,229.109962,281970.0,2407.847760,-0.111,0.123,-0.057,-1.864,1.662,-2.468,3.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48461,10982,439.24,sample_26_10982.wav,sample_26_10982.jpg,Laser-off,Laser-off,NaN,26,Laser-off,0.011000,...,0.000000,0.0,0.000000,0.000,0.000,0.000,-0.076,0.025,0.000,0.080
48462,10983,439.28,sample_26_10983.wav,sample_26_10983.jpg,Laser-off,Laser-off,NaN,26,Laser-off,0.008347,...,0.000000,0.0,0.000000,0.000,0.000,0.000,-0.076,0.025,0.000,0.080
48463,10984,439.32,sample_26_10984.wav,sample_26_10984.jpg,Laser-off,Laser-off,NaN,26,Laser-off,0.011081,...,0.000000,0.0,0.000000,0.000,0.000,0.000,-0.076,0.025,0.000,0.080
48464,10985,439.36,sample_26_10985.wav,sample_26_10985.jpg,Laser-off,Laser-off,NaN,26,Laser-off,0.011044,...,0.000000,0.0,0.000000,0.000,0.000,0.000,-0.076,0.025,0.000,0.080


In [7]:
# Extract labels
y = df_multimodal['class_name_v3'].to_list()
y_array = np.array(y)
le = LabelEncoder()
le.fit(y)
Y_encoded = le.transform(y)

# Non-feature columns to be dropped
non_feature_columns = ['sample index', 'Time_Stamps', 'audio_file_name', 'image_file_name',
                       'class_name', 'class_name_v2', 'class_name_v3', 'Layer number', 'Sample number', 'X', 'Y', 'Z', 'Vx', 'Vy', 'Vz', 'Speed']

# Extract features for the full dataset
X_all = df_multimodal.drop(columns=non_feature_columns).to_numpy()

# Redundant audio features to be dropped
redundant_audio_features = ['dynamic_complexity','loudness']

# Extract audio features
audio_feature_columns = ['rms_energy', 'amplitude_envelope_mean',
                           'amplitude_envelope_std', 'zero_crossing_rate', 'dynamic_complexity',
                           'loudness', 'loudness_vickers', 'spectral_centroid_mean',
                           'spectral_centroid_std', 'spectral_complexity_mean',
                           'spectral_complexity_std', 'spectral_contrast_0_mean',
                           'spectral_contrast_0_std', 'spectral_contrast_1_mean',
                           'spectral_contrast_1_std', 'spectral_contrast_2_mean',
                           'spectral_contrast_2_std', 'spectral_contrast_3_mean',
                           'spectral_contrast_3_std', 'spectral_contrast_4_mean',
                           'spectral_contrast_4_std', 'spectral_contrast_5_mean',
                           'spectral_contrast_5_std', 'spectral_valley_0_mean',
                           'spectral_valley_0_std', 'spectral_valley_1_mean',
                           'spectral_valley_1_std', 'spectral_valley_2_mean',
                           'spectral_valley_2_std', 'spectral_valley_3_mean',
                           'spectral_valley_3_std', 'spectral_valley_4_mean',
                           'spectral_valley_4_std', 'spectral_valley_5_mean',
                           'spectral_valley_5_std', 'spectral_decrease_mean',
                           'spectral_decrease_std', 'spectral_energy_mean', 'spectral_energy_std',
                           'spectral_energy_band_ratio_mean', 'spectral_energy_band_ratio_std',
                           'spectral_flatness_mean', 'spectral_flatness_std', 'spectral_flux_mean',
                           'spectral_flux_std', 'spectral_rolloff_mean', 'spectral_rolloff_std',
                           'spectral_strong_peak_mean', 'spectral_strong_peak_std',
                           'spectral_variance_mean', 'spectral_variance_std',
                           'spectral_skewness_mean', 'spectral_skewness_std',
                           'spectral_kurtosis_mean', 'spectral_kurtosis_std',
                           'spectral_crest_factor_mean', 'spectral_crest_factor_std',
                           'mfcc_0_mean', 'mfcc_0_std', 'mfcc_1_mean', 'mfcc_1_std', 'mfcc_2_mean',
                           'mfcc_2_std', 'mfcc_3_mean', 'mfcc_3_std', 'mfcc_4_mean', 'mfcc_4_std',
                           'mfcc_5_mean', 'mfcc_5_std', 'mfcc_6_mean', 'mfcc_6_std', 'mfcc_7_mean',
                           'mfcc_7_std', 'mfcc_8_mean', 'mfcc_8_std', 'mfcc_9_mean', 'mfcc_9_std',
                           'mfcc_10_mean', 'mfcc_10_std', 'mfcc_11_mean', 'mfcc_11_std',
                           'mfcc_12_mean', 'mfcc_12_std']

X_audio = df_multimodal[audio_feature_columns].drop(columns=redundant_audio_features).to_numpy()

# Extract vision features
vision_feature_columns = ['max_contour_area', 'rectangle_angle',
                          'rectangle_width', 'rectangle_height', 'ellipse_angle', 'ellipse_width',
                          'ellipse_height', 'max_hull', 'm00', 'm10', 'm01', 'm20', 'm11', 'm02',
                          'm30', 'm21', 'm12', 'm03', 'mu20', 'mu11', 'mu02', 'mu30', 'mu21',
                          'mu12', 'mu03', 'nu20', 'nu11', 'nu02', 'nu30', 'nu21', 'nu12', 'nu03',
                          'center_x', 'center_y', 'contour_area', 'contour_length']

X_vision = df_multimodal[vision_feature_columns].to_numpy()

X_all.shape, X_audio.shape, X_vision.shape

((36557, 119), (36557, 81), (36557, 36))

In [8]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(y),
                                                 y = y)

class_weights_encoded = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(Y_encoded),
                                                 y = Y_encoded)


class_names = (le.classes_).tolist()
print(f"class weights: {class_weights}")
print(f"class weights encoded: {class_weights_encoded}")
print(f"class        : {class_names}")

class weights: [1.20237469 1.83114606 1.472173   0.51468435]
class weights encoded: [1.20237469 1.83114606 1.472173   0.51468435]
class        : ['Defect-free', 'Keyhole pores', 'Laser defocus', 'Laser-off']


In [9]:
# Given features based on previous analysis results
features_dropped_090 = ['loudness_vickers', 'spectral_centroid_mean',
       'spectral_centroid_std', 'spectral_complexity_mean',
       'spectral_complexity_std', 'spectral_contrast_1_mean',
       'spectral_contrast_3_mean', 'spectral_contrast_3_std',
       'spectral_contrast_4_mean', 'spectral_contrast_5_mean',
       'spectral_valley_0_mean', 'spectral_valley_2_mean',
       'spectral_valley_3_mean', 'spectral_valley_4_mean',
       'spectral_valley_5_mean', 'spectral_energy_band_ratio_std',
       'spectral_flux_mean', 'spectral_flux_std', 'spectral_rolloff_mean',
       'spectral_strong_peak_mean', 'spectral_kurtosis_mean',
       'spectral_crest_factor_mean', 'spectral_crest_factor_std',
       'mfcc_1_mean', 'mfcc_2_mean', 'mfcc_3_mean', 'mfcc_4_std',
       'mfcc_6_mean', 'max_contour_area', 'rectangle_angle', 'ellipse_angle',
       'nu02', 'nu03']

features_dropped_090_mi_filtered_01 = ['loudness_vickers', 'spectral_complexity_mean',
       'spectral_complexity_std', 'spectral_contrast_1_mean',
       'spectral_contrast_3_mean', 'spectral_valley_0_mean',
       'spectral_valley_2_mean', 'spectral_valley_3_mean',
       'spectral_valley_4_mean', 'spectral_valley_5_mean',
       'spectral_flux_mean', 'spectral_flux_std', 'spectral_rolloff_mean',
       'spectral_kurtosis_mean', 'mfcc_1_mean', 'mfcc_3_mean',
       'max_contour_area', 'rectangle_angle', 'ellipse_angle', 'nu02', 'nu03']

features_dropped_090_mi_filtered_005 = ['loudness_vickers', 'spectral_centroid_mean',
       'spectral_centroid_std', 'spectral_complexity_mean',
       'spectral_complexity_std', 'spectral_contrast_1_mean',
       'spectral_contrast_3_mean', 'spectral_contrast_3_std',
       'spectral_contrast_4_mean', 'spectral_contrast_5_mean',
       'spectral_valley_0_mean', 'spectral_valley_2_mean',
       'spectral_valley_3_mean', 'spectral_valley_4_mean',
       'spectral_valley_5_mean', 'spectral_energy_band_ratio_std',
       'spectral_flux_mean', 'spectral_flux_std', 'spectral_rolloff_mean',
       'spectral_strong_peak_mean', 'spectral_kurtosis_mean',
       'spectral_crest_factor_mean', 'spectral_crest_factor_std',
       'mfcc_1_mean', 'mfcc_2_mean', 'mfcc_3_mean', 'mfcc_4_std',
       'mfcc_6_mean', 'max_contour_area', 'rectangle_angle', 'ellipse_angle',
       'nu02', 'nu03']

# Create ablated datasets
X_dropped_090 = df_multimodal[features_dropped_090].to_numpy()
X_dropped_090_mi_filtered_01 = df_multimodal[features_dropped_090_mi_filtered_01].to_numpy()
X_dropped_090_mi_filtered_005 = df_multimodal[features_dropped_090_mi_filtered_005].to_numpy()

# Extract Audio and Vision features from the ablated datasets
X_audio_dropped_090 = df_multimodal[[feat for feat in features_dropped_090 if feat in audio_feature_columns]].to_numpy()
X_vision_dropped_090 = df_multimodal[[feat for feat in features_dropped_090 if feat in vision_feature_columns]].to_numpy()

# Extract Audio features from the ablated datasets 2 and 3
X_audio_dropped_090_mi_filtered_01 = df_multimodal[[feat for feat in features_dropped_090_mi_filtered_01 if feat in audio_feature_columns]].to_numpy()
X_audio_dropped_090_mi_filtered_005 = df_multimodal[[feat for feat in features_dropped_090_mi_filtered_005 if feat in audio_feature_columns]].to_numpy()

# Get the shapes of these additional ablated datasets
X_dropped_090.shape, X_dropped_090_mi_filtered_01.shape, X_dropped_090_mi_filtered_005.shape, X_audio_dropped_090.shape, X_vision_dropped_090.shape, X_audio_dropped_090_mi_filtered_01.shape, X_audio_dropped_090_mi_filtered_005.shape

((36557, 33),
 (36557, 21),
 (36557, 33),
 (36557, 28),
 (36557, 5),
 (36557, 16),
 (36557, 28))

- ALL Dataset (X_all):  `119 features`
- Ablated Dataset 1 (X_dropped_090):  `33 features`
- Ablated Dataset 2 (X_dropped_090_mi_filtered_01): `21 features`
- Ablated Dataset 3 (X_dropped_090_mi_filtered_005):  `33 features`
- Ablated Dataset 4 (X_audio_all): `81 audio features`
- Ablated Dataset 5 (X_vision_all): `36 vision features`
- Ablated Dataset 6 (X_audio_dropped_090): `28 audio features`
- Ablated Dataset 7 (X_vision_dropped_090): `5 vision features`
- Ablated Dataset 8 (X_audio_dropped_090_mi_filtered_01): `16 audio features`
- Ablated Dataset 9 (X_audio_dropped_090_mi_filtered_005): `28 audio features`

In [10]:
%store -r train_indices
%store -r val_indices
%store -r test_indices

In [11]:
len(test_indices), len(train_indices)

(462, 3692)

## Step 2: Create a Pipeline and Iterate Over Datasets and Models

In [12]:
# Define hyperparameters for each model
hyperparameters = {
    'KNN': {'n_neighbors': 12, 'weights': 'distance'},
    'DT': {'ccp_alpha': 3.89e-05, 'max_depth': 21, 'min_samples_leaf': 1, 'min_samples_split': 13},
    'LR': {'C': 5.15, 'max_iter': 2528, 'penalty': 'l2', 'solver': 'lbfgs'},
    'RF': {'bootstrap': False, 'max_depth': 64, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 6, 'n_estimators': 307},
    'BalancedRF': {'bootstrap': False, 'max_depth': 23, 'max_features': 'sqrt', 'min_samples_leaf': 8, 'min_samples_split': 17, 'n_estimators': 237},
    'RUSBoost': {'learning_rate': 0.1, 'n_estimators': 150},
    'Ada': {'algorithm': 'SAMME', 'learning_rate': 0.57, 'n_estimators': 307},
    'GP': {'kernel': 1**2 * RBF(length_scale=1)},
    'NN': {'activation': 'relu', 'alpha': 0.077, 'early_stopping': True, 'hidden_layer_sizes': (229, 255, 168), 'learning_rate': 'adaptive', 'max_iter': 11805, 'solver': 'adam', 'validation_fraction': 0.1},
    'XGBoost': {'colsample_bytree': 0.62, 'gamma': 0.48, 'learning_rate': 0.35, 'max_depth': 44, 'min_child_weight': 8, 'n_estimators': 480, 'subsample': 0.59,  'gpu_id': 0},
    'LightGBM': {'colsample_bytree': 0.80, 'learning_rate': 0.15, 'max_depth': 35, 'min_child_weight': 1, 'n_estimators': 276, 'num_leaves': 120, 'reg_alpha': 1.94, 'reg_lambda': 1.70, 'subsample': 0.86, 'verbose': -1},
    'SVM': {'C': 524.86, 'degree': 9, 'gamma': 0.029, 'kernel': 'rbf', 'probability': True},
    'GB': {'learning_rate': 0.23, 'max_depth': 49, 'min_samples_leaf': 10, 'min_samples_split': 17, 'n_estimators': 271, 'n_iter_no_change': 10, 'subsample': 0.73, 'tol': 0.0001},
    'HistGB': {'learning_rate': 0.23, 'max_iter': 100, 'max_depth': 49, 'min_samples_leaf': 10, 'n_iter_no_change': 10, 'tol': 0.0001},
}

# List of datasets
datasets = {
    'All Features': X_all,
    'Ablated 1': X_dropped_090,
    'Ablated 2': X_dropped_090_mi_filtered_01,
    'Ablated 3': X_dropped_090_mi_filtered_005,
    'Audio All': X_audio,
    'Vision All': X_vision,
    'Audio Ablated 1': X_audio_dropped_090,
    'Vision Ablated 1': X_vision_dropped_090,
    'Audio Ablated 2': X_audio_dropped_090_mi_filtered_01,
    'Audio Ablated 3': X_audio_dropped_090_mi_filtered_005
}


# List of models
models = {
    'KNN': KNeighborsClassifier,
    'DT': DecisionTreeClassifier,
    # 'LR': LogisticRegression,
    'RF': RandomForestClassifier,
    'BalancedRF': BalancedRandomForestClassifier,
    'RUSBoost': RUSBoostClassifier,
    'Ada': AdaBoostClassifier,
    # 'GP': GaussianProcessClassifier,
    'NN': MLPClassifier,
    'XGBoost': XGBClassifier,
    'LightGBM': LGBMClassifier,
    'SVM': SVC,
    # 'GB': GradientBoostingClassifier,
    'HistGB': HistGradientBoostingClassifier
}

## Step 3: Model Training

In [13]:
# Initialize KFold with the number of splits you desire
kf = KFold(n_splits=5)
# Create a dictionary to store final results
final_results = defaultdict(list)
# Create a dictionary to store temporary cross-validation results for each dataset and model
cv_results = defaultdict(lambda: defaultdict(list))


# Iterate over datasets and models
for dataset_name, X in datasets.items():
    print(f"Processing dataset: {dataset_name}")
    # Split data
    X_train = X[train_indices]
    X_val = X[val_indices]
    X_test = X[test_indices]
  
    y_train = Y_encoded[train_indices]
    y_val = Y_encoded[val_indices]
    y_test = Y_encoded[test_indices]

    # Scaling
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    with open(os.path.join(scaler_path, f'StandardScaler_{dataset_name}.pkl'), 'wb') as file:
        pickle.dump(scaler, file)

    for model_name, Model in models.items():
        print(f"\tTraining model: {model_name}")
        with wandb.init(project="AudioVisual_ML_ablation_study_v2", name=f"{model_name}_{dataset_name}", reinit=True) as run:
            # Log model and hyperparameters
            wandb.config.update({"Model": model_name, "Hyperparameters": hyperparameters[model_name]})
    
            # Initialize model with the best hyperparameters
            model = Model(**hyperparameters[model_name])
    
            # If the model is SVM, use only 20% of the training data
            if model_name == 'SVM':
                sample_indices = np.random.choice(len(X_train), int(0.2 * len(X_train)), replace=False)
                X_train_sampled = X_train[sample_indices]
                y_train_sampled = y_train[sample_indices]
                model.fit(X_train_sampled, y_train_sampled)
            else:
                # Train the model
                model.fit(X_train, y_train)

            # Save the model
            dump(model, os.path.join(model_path, f'{model_name}_{dataset_name}.joblib'))


            #####------   cross-validation on the testing dataset--------------
            fold_metrics = defaultdict(list)
    
            for train_idx, test_idx in kf.split(X_test):
                X_test_train, X_test_val = X_test[train_idx], X_test[test_idx]
                y_test_train, y_test_val = y_test[train_idx], y_test[test_idx]
                
                # Make predictions
                y_test_pred = model.predict(X_test_val)
                
                # Evaluate the model
                acc = accuracy_score(y_test_val, y_test_pred)
                f1 = f1_score(y_test_val, y_test_pred, average='weighted')
                precision = precision_score(y_test_val, y_test_pred, average='weighted')
                recall = recall_score(y_test_val, y_test_pred, average='weighted')
                
                # Optional: ROC-AUC can only be calculated for binary classification or multilabel indicator format
                try:
                    roc_auc = roc_auc_score(y_test_val, model.predict_proba(X_test_val), multi_class='ovr', average='weighted')
                    
                except:
                    roc_auc = 'N/A'
                
                # Store the metrics for this fold
                fold_metrics['Accuracy'].append(acc)
                fold_metrics['F1 Score'].append(f1)
                fold_metrics['Precision'].append(precision)
                fold_metrics['Recall'].append(recall)
                fold_metrics['ROC-AUC'].append(roc_auc if roc_auc != 'N/A' else np.nan)
            
            # Calculate mean and std for each metric and print them
            for metric, values in fold_metrics.items():
                mean_val = np.nanmean(values)  # Handle 'N/A' values
                std_val = np.nanstd(values)    # Handle 'N/A' values
                
                print(f"\t\t{metric}: Mean = {mean_val}, Std = {std_val}")
                # Log the mean and std values to wandb
                wandb.log({f"{metric}_mean": mean_val, f"{metric}_std": std_val})
                
                cv_results[dataset_name][f"{model_name}_{metric}_mean"] = mean_val
                cv_results[dataset_name][f"{model_name}_{metric}_std"] = std_val
             
            #####------   cross-validation on the testing dataset--------------
            
# Convert the temporary results to final format
for dataset_name, metrics in cv_results.items():
    final_results['Dataset'].append(dataset_name)
    for metric_name, value in metrics.items():
        final_results[metric_name].append(value)

# Convert results to a DataFrame for easier viewing
final_results_df = pd.DataFrame(final_results)

Processing dataset: All Features
	Training model: KNN


		Accuracy: Mean = 0.8701963534361852, Std = 0.04364919371953449
		F1 Score: Mean = 0.872004096850047, Std = 0.04237379402456881
		Precision: Mean = 0.8787829247671806, Std = 0.03908269162088463
		Recall: Mean = 0.8701963534361852, Std = 0.04364919371953449
		ROC-AUC: Mean = 0.9692253336554998, Std = 0.013861821271084847


Accuracy_mean,▁
Accuracy_std,▁
F1 Score_mean,▁
F1 Score_std,▁
Precision_mean,▁
Precision_std,▁
ROC-AUC_mean,▁
ROC-AUC_std,▁
Recall_mean,▁
Recall_std,▁
Accuracy_mean,0.8702


	Training model: DT


		Accuracy: Mean = 0.8442730247779335, Std = 0.04490565142150513
		F1 Score: Mean = 0.8458100105448153, Std = 0.044475053765800146
		Precision: Mean = 0.8504462469735087, Std = 0.0442976986779252
		Recall: Mean = 0.8442730247779335, Std = 0.04490565142150513
		ROC-AUC: Mean = 0.8995609118256727, Std = 0.0314111711366799


...

Accuracy_mean,▁
Accuracy_std,▁
F1 Score_mean,▁
F1 Score_std,▁
Precision_mean,▁
Precision_std,▁
ROC-AUC_mean,▁
ROC-AUC_std,▁
Recall_mean,▁
Recall_std,▁
Accuracy_mean,0.79883


	Training model: HistGB


		Accuracy: Mean = 0.8420757363253857, Std = 0.03345472670594328
		F1 Score: Mean = 0.8326945505052553, Std = 0.030519909870100632
		Precision: Mean = 0.8326716494869892, Std = 0.03408887887926169
		Recall: Mean = 0.8420757363253857, Std = 0.03345472670594328
		ROC-AUC: Mean = 0.9636598126851691, Std = 0.012910291526973178


/home/chenlequn/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy_mean,▁
Accuracy_std,▁
F1 Score_mean,▁
F1 Score_std,▁
Precision_mean,▁
Precision_std,▁
ROC-AUC_mean,▁
ROC-AUC_std,▁
Recall_mean,▁
Recall_std,▁
Accuracy_mean,0.84208


In [14]:
%store datasets
%store le
%store Y_encoded

Stored 'datasets' (dict)
Stored 'le' (LabelEncoder)
Stored 'Y_encoded' (ndarray)
